# GRASS GIS Notebook for Tools for Open Geospatial Science Course

This is a quick introduction into Jupyter Notebook. This is a first cell and besides a heading it
contains text. The syntax used is Markdown.

Python code can be written like this and executed using Shift+Enter:

In [ ]:
# This cell gives is a quick introduction into Jupyter Notebook.
# Python code is written as expected:
a = 6
b = 7
c = a * b

# We are using Python 2, so print is a statement by default:
print "Answer is", c

# In Jupyter Notebook, Python code can be mixed with command line code (Bash).
# It is enough just to prefix the command line with an exclamation mark:
!echo "Answer is $c"

# Use Shift+Enter to execute this cell. The result is below.

In [ ]:
# download a sample dataset
!mkdir -p /home/jovyan/grassdata \
  && curl -SL https://grass.osgeo.org/sampledata/north_carolina/nc_basic_spm_grass7.zip > nc_basic_spm_grass7.zip \
  && unzip -qq nc_basic_spm_grass7.zip \
  && mv nc_basic_spm_grass7 /home/jovyan/grassdata \
  && rm nc_basic_spm_grass7.tar.gz

In [ ]:
import os
import sys
import subprocess
from IPython.display import Image

# create GRASS GIS runtime environment
gisbase = subprocess.check_output(["grass", "--config", "path"]).strip()
os.environ['GISBASE'] = gisbase
sys.path.append(os.path.join(gisbase, "etc", "python"))

# do GRASS GIS imports
import grass.script as gs
import grass.script.setup as gsetup

# set GRASS GIS session data
rcfile = gsetup.init(gisbase, "/home/jovyan/grassdata", "nc_basic_spm_grass7", "user1")

# default font displays
os.environ['GRASS_FONT'] = 'sans'
# overwrite existing maps
os.environ['GRASS_OVERWRITE'] = '1'
gs.set_raise_on_error(True)
gs.set_capture_stderr(True)

# set display modules to render into a file (named map.png by default)
os.environ['GRASS_RENDER_IMMEDIATE'] = 'cairo'
os.environ['GRASS_RENDER_FILE_READ'] = 'TRUE'
os.environ['GRASS_LEGEND_FILE'] = 'legend.txt'

In [ ]:
!g.list vect

### Raster buffer

Set computational region and create multiple buffers in given distances
around lakes represented as raster:

In [ ]:
gs.parse_command('g.region', raster="lakes", flags='pg')
gs.run_command('r.buffer', input="lakes", output="lakes_buff", distances=[60, 120, 240, 500])
gs.run_command('d.erase')
gs.run_command('d.rast', map="lakes_buff")
gs.run_command('d.legend', raster="lakes_buff", range=(2, 5), at=(80, 100, 2, 10))
Image(filename="map.png")

## Vector buffer

Create a negative buffer around state boundary represented as a vector.
Vector modules typically don't follow computational region,
but we set it to inform display modules about our area of interest.

In [ ]:
gs.run_command('v.buffer', input="boundary_state", output="buffer", distance=-10000)
gs.parse_command('g.region', vector="boundary_state", flags='pg')
gs.run_command('d.erase')
!rm -f $GRASS_LEGEND_FILE
gs.run_command('d.vect', map="boundary_state", fill_color="#5A91ED", legend_label="State boundary")
gs.run_command('d.vect', map="buffer", fill_color="#F8766D", legend_label="Inner portion")
gs.run_command('d.legend.vect', at=(10, 35))
Image(filename="map.png")